In [5]:
import numpy as np
import pandas as pd
import random

from collections import Counter

import decision_tree

In [4]:
df = pd.read_csv("../datasets/drug200.csv")

# get unique values for each column
for col in df.columns:
    print(col, df[col].unique())
    print()

Age [23 47 28 61 22 49 41 60 43 34 74 50 16 69 32 57 63 48 33 31 39 45 18 65
 53 46 15 73 58 66 37 68 67 62 24 26 40 38 29 17 54 70 36 19 64 59 51 42
 56 20 72 35 52 55 30 21 25]

Sex ['F' 'M']

BP ['HIGH' 'LOW' 'NORMAL']

Cholesterol ['HIGH' 'NORMAL']

Na_to_K [25.355 13.093 10.114  7.798 18.043  8.607 16.275 11.037 15.171 19.368
 11.767 19.199 15.376 20.942 12.703 15.516 11.455 13.972  7.298 25.974
 19.128 25.917 30.568 15.036 33.486 18.809 30.366  9.381 22.697 17.951
  8.75   9.567 11.014 31.876 14.133  7.285  9.445 13.938  9.709  9.084
 19.221 14.239 15.79  12.26  12.295  8.107 13.091 10.291 31.686 19.796
 19.416 10.898 27.183 18.457 10.189 14.16  11.34  27.826 10.091 18.703
 29.875  9.475 20.693  8.37  13.303 27.05  12.856 10.832 24.658 24.276
 13.967 19.675 10.605 22.905 17.069 20.909 11.198 19.161 13.313 10.84
 13.934  7.761  9.712 11.326 10.067 13.935 13.597 15.478 23.091 17.211
 16.594 15.156 29.45  29.271 15.015 11.424 38.247 25.395 35.639 16.725
 11.871 12.854 13.127  8.966 

In [6]:
def create_dataset():

    # Set the random seed for reproducibility
    random.seed(42)

    # Define the number of rows
    num_rows = 1500

    # Create a dictionary to store the data for each column
    data = {
        'chest_pain': [random.choice(['Yes', 'Yes', 'No']) for _ in range(num_rows)],
        'good_blood_circulation': [random.choice(['Yes', 'No', 'No']) for _ in range(num_rows)],
        'blocked_arteries': [random.choice(['Yes', 'No', 'No']) for _ in range(num_rows)],
        'overweight': [random.choice(['Yes', 'Yes', 'No']) for _ in range(num_rows)],
        'high_blood_pressure': [random.choice(['Yes', 'Yes', 'Yes', 'No']) for _ in range(num_rows)],
        'heart_disease': [random.choice(['Yes', 'No']) for _ in range(num_rows)]
    }

    # Create a pandas DataFrame from the dictionary
    df = pd.DataFrame(data)

    return df

In [8]:
df = create_dataset()
df

,chest_pain,good_blood_circulation,blocked_arteries,overweight,high_blood_pressure,heart_disease
0,No,No,No,Yes,Yes,No
1,Yes,No,Yes,Yes,Yes,Yes
2,Yes,Yes,No,No,No,No
3,No,No,No,Yes,No,No
4,Yes,No,No,Yes,Yes,No
...,...,...,...,...,...,...
1495,Yes,No,No,No,Yes,No
1496,Yes,Yes,Yes,No,No,No
1497,No,Yes,No,Yes,Yes,No
1498,Yes,Yes,Yes,Yes,Yes,Yes


In [12]:
list = df['heart_disease'].unique()

list[1]

'Yes'

left of by finishing up the best split function 

- next steps:

    - calculate gini impurity before split
    - and then the if if best_gini < gini_without_spit and best_feature_name is not None:

In [ ]:
class Node():
    
    def __init__(self, df : pd.DataFrame, node_type : str):
        
        self.df = df
        self.node_type = node_type
        self.best_feature = None
        self.best_gini = None
        self.class_distribution = None
        self.most_common = None
        

        
    def create_node(self):
        
        best_feature_name, best_gini = self.best_split()
        
        
        
    def best_split(self):
        
        best_gini_value = 1.0  # Initialize the best Gini impurity with a high value
        best_feature_name = None  # Initialize the best feature name as None
        best_feature_value_continuous = None  # Initialize the best feature value for continuous features
        
        column_names = [col for col in self.df.columns if col not in ['target']]
        
        # Iterate through all the features (columns) in the DataFrame
        for feature_name in column_names:
        
            # check if the feature is categorical
            if self.df[feature_name].dtype == 'O':
        
                gini_feature_value = self.calculate_gini_categorical(feature_name)

            elif self.df[feature_name].dtype in ['int64', 'float64']:
                
                gini_feature_value, best_split_value = self.calculate_gini_continuous(feature_name)

            # Check if this is the best split so far (lowest Gini impurity)
            if gini_feature_value < best_gini_value and self.df[feature_name].dtype == 'O':
        
                best_gini_value = gini_feature_value
                best_feature_name = feature_name
                
            elif gini_feature_value < best_gini_value and self.df[feature_name].dtype in ['int64', 'float64']:
                
                best_gini_value = gini_feature_value
                best_feature_name = feature_name
                best_feature_value_continuous = best_split_value
        
        # Return the best feature for splitting and its associated Gini impurity
        if best_feature_value_continuous != None:
            return best_feature_name, best_gini_value, best_feature_value_continuous
        else:
            return best_feature_name, best_gini_value
    
    def calculate_gini_categorical(self, feature_name):
        
        # Count the number of 'Yes' and 'No' labels in the left and right branches
        left_counter = Counter(self.df[self.df[feature_name] == "Yes"]['target'])
        right_counter = Counter(self.df[self.df[feature_name] == "No"]['target'])

        # Extract the counts of 'Yes' and 'No' for left and right branches
        count_left_yes, count_left_no = self.get_counts(left_counter)
        count_right_yes, count_right_no = self.get_counts(right_counter)
        
        left_gini_impurity = self.calculate_gini_impurity(count_left_yes, count_left_no)
        right_gini_impurity = self.calculate_gini_impurity(count_right_yes, count_right_no)
        
        # Calculate the Gini impurity for the split
        gini_impurity_feature = self.calculate_gini_impurity_split(left_gini_impurity, right_gini_impurity, count_left_yes, count_left_no, count_right_yes, count_right_no)
        
        return gini_impurity_feature
        
    def calculate_gini_continuous(self, feature_name):
        
        # Get the unique values in the feature
        unique_values = self.df[feature_name].unique()
        
        # Sort the unique values
        unique_values.sort()
        
        best_gini_temp = 1.0
        best_split_value = None

        # Iterate through all the unique values
        for i in range(1, len(unique_values)):
            
            # Calculate the potential split value
            split_value = (unique_values[i - 1] + unique_values[i]) / 2
            
            # Split the DataFrame into two subsets
            left_subset = self.df[self.df[feature_name] <= split_value]
            right_subset = self.df[self.df[feature_name] > split_value]
            
            # Calculate the Gini impurity for the split
            gini = self.calculate_gini_continuous_split(left_subset, right_subset)
            
            # Check if this is the best split so far
            if gini < best_gini:
                best_gini = gini
                best_split_value = split_value
        
        return best_gini, best_split_value
    
    def calculate_gini_impurity(self, count_yes, count_no):
        
        # Calculate the total number of instances
        total = count_yes + count_no

        if total == 0:
            return 0
        else:
            # Calculate the Gini impurity
            return 1 - (count_yes / total)**2 - (count_no / total)**2 # 1 - (prob_yes**2 + prob_no**2)
        
    def calculate_gini_impurity_split(self, left_gini_impurity, right_gini_impurity, count_left_yes, count_left_no, count_right_yes, count_right_no):
        
        # Calculate the total number of instances
        total = count_left_yes + count_left_no + count_right_yes + count_right_no
        
        # Calculate the Gini impurity for the split
        return ( (count_left_yes + count_left_no) / total * left_gini_impurity ) + ( (count_right_yes + count_right_no) / total * right_gini_impurity )

In [ ]:
class DecisionTree():
    
    def __intit__(self, max_depth : int = None, num_min_samples : int = 20):
        
        # function to check the parameters
        # TODO
        
        self.max_depth = max_depth  
        self.num_min_samples = num_min_samples  
        self.depth = 0
        self.nodes = []
        
    def fit(self, X : np.ndarray, y : np.ndarray):
        
        # function to check the parameters
        # TODO
        
        self.df = X.copy()
        self.df['target'] = y.copy()
        
        
    def grow_tree(self):
        
        if self.depth < self.max_depth and len(self.df) > self.num_min_samples:
            
            if self.depth == 0:
                
                root_node = Node(self.df, 'root')
                root_node.create_node()
            else:
                
        
        
        
    def predict(self, X_test):
        return self.root.predict(X_test)        
    
    
